# 02: Intermediate Text-to-SQL with Dynamic Table Retrieval

Welcome to the second notebook! In this notebook, you'll learn how to scale text-to-SQL to larger databases using dynamic table retrieval.

## Learning Objectives

By the end of this notebook, you will be able to:
- Work with multi-table databases
- Use `SQLTableRetrieverQueryEngine` for large schemas
- Implement dynamic table retrieval with ObjectIndex
- Understand when to use different query engines
- Query CSV files directly with DuckDB

## What You'll Learn

In Notebook 01, we sent all table schemas to the LLM with every query. This works for small databases but doesn't scale to databases with many tables. In this notebook, you'll learn how to use **vector similarity** to dynamically retrieve only the most relevant tables for each query.

## Dynamic Table Retrieval

When you have many tables, sending all schemas to the LLM becomes impractical. Dynamic table retrieval uses vector similarity to find the most relevant tables for each query.

**Problem:** Database has 50 tables, but LLM context limit is 100K tokens  
**Solution:** Retrieve only the 2-3 most relevant tables based on the query

## Section 1: Setup and Configuration

### 1.1 Import Required Libraries

In [2]:
# Standard library imports
import os
from pathlib import Path
from typing import List

# Third-party imports
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np

# LlamaIndex imports
from llama_index.core import SQLDatabase, VectorStoreIndex
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


### 1.2 Load Environment and Initialize LLM

In [3]:
# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found. Please check your .env file")

# Initialize LLM
llm = OpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
    api_key=OPENAI_API_KEY
)

print("✓ Environment loaded and LLM initialized")
print(f"  Model: {llm.model}")

✓ Environment loaded and LLM initialized
  Model: gpt-4o-mini


### 1.3 Create SQLite Database

We'll use SQLite for the main tutorial because it has excellent compatibility with LlamaIndex's SQLDatabase class. In Section 5, we'll demonstrate DuckDB's powerful features for querying CSV files directly.

In [4]:
# Create SQLite database (compatible with LlamaIndex SQLDatabase)
# Note: We use SQLite instead of DuckDB for the main tutorial because SQLite
# has better compatibility with SQLAlchemy's reflection system used by LlamaIndex.
# We'll still demonstrate DuckDB's powerful features in Section 5!

engine = create_engine("sqlite:///ecommerce.db")

# Test connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1 as test"))
    test_value = result.fetchone()[0]
    print(f"✓ SQLite connection successful (test value: {test_value})")

print(f"  Database: ecommerce.db")
print(f"  Dialect: {engine.dialect.name}")

✓ SQLite connection successful (test value: 1)
  Database: ecommerce.db
  Dialect: sqlite


## Section 2: Create Multi-Table E-Commerce Dataset

We'll create a realistic e-commerce database with multiple related tables. This demonstrates the challenge: with many tables, sending all schemas to the LLM becomes impractical.

### 2.1 Generate Sample Data

In [5]:
# Create customers table
np.random.seed(42)

# Generate customers
num_customers = 100
customers_data = {
    'customer_id': range(1, num_customers + 1),
    'name': [f'Customer_{i}' for i in range(1, num_customers + 1)],
    'email': [f'customer{i}@example.com' for i in range(1, num_customers + 1)],
    'region': np.random.choice(['North', 'South', 'East', 'West'], num_customers),
    'signup_date': pd.date_range('2020-01-01', periods=num_customers, freq='3D')
}

customers_df = pd.DataFrame(customers_data)
print("✓ Customers table created")
print(f"  Shape: {customers_df.shape}")
print(customers_df.head())

✓ Customers table created
  Shape: (100, 5)
   customer_id        name                  email region signup_date
0            1  Customer_1  customer1@example.com   East  2020-01-01
1            2  Customer_2  customer2@example.com   West  2020-01-04
2            3  Customer_3  customer3@example.com  North  2020-01-07
3            4  Customer_4  customer4@example.com   East  2020-01-10
4            5  Customer_5  customer5@example.com   East  2020-01-13


In [6]:
# Generate products
categories = ['Electronics', 'Clothing', 'Food', 'Books', 'Home']
num_products = 50

products_data = {
    'product_id': range(1, num_products + 1),
    'product_name': [f'Product_{i}' for i in range(1, num_products + 1)],
    'category': np.random.choice(categories, num_products),
    'price': np.round(np.random.uniform(10, 500, num_products), 2),
    'stock': np.random.randint(0, 100, num_products)
}

products_df = pd.DataFrame(products_data)
print("✓ Products table created")
print(f"  Shape: {products_df.shape}")
print(products_df.head())

✓ Products table created
  Shape: (50, 5)
   product_id product_name  category   price  stock
0           1    Product_1  Clothing  486.14     11
1           2    Product_2  Clothing  425.97     38
2           3    Product_3     Books  363.65      1
3           4    Product_4  Clothing  125.63      2
4           5    Product_5  Clothing  135.47     55


In [9]:
# Generate orders
num_orders = 500

orders_data = {
    'order_id': range(1, num_orders + 1),
    'customer_id': np.random.randint(1, num_customers + 1, num_orders),
    'product_id': np.random.randint(1, num_products + 1, num_orders),
    'quantity': np.random.randint(1, 10, num_orders),
    'order_date': pd.date_range('2020-01-01', periods=num_orders, freq='1D'),
    'status': np.random.choice(['Pending', 'Shipped', 'Delivered', 'Cancelled'], num_orders, p=[0.1, 0.3, 0.5, 0.1])
}

orders_df = pd.DataFrame(orders_data)

# Add order total based on product price and quantity
orders_df = orders_df.merge(products_df[['product_id', 'price']], on='product_id')
orders_df['order_total'] = orders_df['quantity'] * orders_df['price']
orders_df = orders_df.drop('price', axis=1)

print("✓ Orders table created")
print(f"  Shape: {orders_df.shape}")
print(orders_df.head())

✓ Orders table created
  Shape: (500, 7)
   order_id  customer_id  product_id  quantity order_date     status  \
0         1           67           5         5 2020-01-01  Delivered   
1         2           96          20         5 2020-01-02  Delivered   
2         3           67          11         4 2020-01-03    Shipped   
3         4           27          42         6 2020-01-04  Cancelled   
4         5           93           2         3 2020-01-05  Delivered   

   order_total  
0       677.35  
1      1517.20  
2      1795.68  
3      2095.74  
4      1277.91  


In [10]:
# Generate product reviews
num_reviews = 200

reviews_data = {
    'review_id': range(1, num_reviews + 1),
    'product_id': np.random.randint(1, num_products + 1, num_reviews),
    'customer_id': np.random.randint(1, num_customers + 1, num_reviews),
    'rating': np.random.randint(1, 6, num_reviews),
    'review_date': pd.date_range('2020-06-01', periods=num_reviews, freq='2D')
}

reviews_df = pd.DataFrame(reviews_data)
print("✓ Reviews table created")
print(f"  Shape: {reviews_df.shape}")
print(reviews_df.head())

✓ Reviews table created
  Shape: (200, 5)
   review_id  product_id  customer_id  rating review_date
0          1          34           99       3  2020-06-01
1          2          41           40       3  2020-06-03
2          3          47           14       3  2020-06-05
3          4          18           35       5  2020-06-07
4          5          11           48       4  2020-06-09


### 2.2 Load Data into Database

In [11]:
# Save DataFrames as CSV for Section 5 demo (DuckDB can query CSV files directly)
customers_df.to_csv('customers.csv', index=False)
products_df.to_csv('products.csv', index=False)
orders_df.to_csv('orders.csv', index=False)
reviews_df.to_csv('reviews.csv', index=False)

print("✓ CSV files created (for Section 5 demo)")

✓ CSV files created (for Section 5 demo)


In [13]:
# Load DataFrames into SQLite using pandas to_sql()
with engine.connect() as conn:
    customers_df.to_sql('customers', conn, if_exists='replace', index=False)
    products_df.to_sql('products', conn, if_exists='replace', index=False)
    orders_df.to_sql('orders', conn, if_exists='replace', index=False)
    reviews_df.to_sql('reviews', conn, if_exists='replace', index=False)
    conn.commit()

print("✓ Tables loaded into SQLite")

# Verify tables (SQLite syntax)
with engine.connect() as conn:
    result = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table'"))
    tables = [row[0] for row in result]
    print(f"  Tables: {tables}")

✓ Tables loaded into SQLite
  Tables: ['customers', 'products', 'orders', 'reviews']


### 2.3 Inspect Schema

Now we'll create a SQLDatabase object that LlamaIndex can use to query our tables.

In [15]:
# Create SQLDatabase object
sql_database = SQLDatabase(
    engine,
    include_tables=["customers", "products", "orders", "reviews"]
)

print("✓ SQLDatabase created successfully")
print("  Tables: customers, products, orders, reviews")

✓ SQLDatabase created successfully
  Tables: customers, products, orders, reviews


## Section 3: ObjectIndex for Table Schemas

Now we'll create an ObjectIndex that stores table schemas and allows semantic retrieval.

### 3.1 Create SQLTableSchema Objects

In [16]:
# Define table schemas with descriptive context
# The context_str helps the retriever understand when to use each table
table_schema_objs = [
    SQLTableSchema(
        table_name="customers",
        context_str="""
        Contains customer information including customer_id, name, email, region, and signup_date.
        Use this table for queries about customer demographics, contact information, 
        customer segmentation by region, or customer acquisition over time.
        """
    ),
    SQLTableSchema(
        table_name="products",
        context_str="""
        Contains product catalog with product_id, product_name, category, price, and stock.
        Use this table for queries about product information, pricing, inventory levels,
        product categories, or available products.
        """
    ),
    SQLTableSchema(
        table_name="orders",
        context_str="""
        Contains order transactions with order_id, customer_id, product_id, quantity, 
        order_date, status, and order_total.
        Use this table for queries about sales, revenue, order volumes, order status,
        purchase patterns, or transaction analysis.
        """
    ),
    SQLTableSchema(
        table_name="reviews",
        context_str="""
        Contains product reviews with review_id, product_id, customer_id, rating, and review_date.
        Use this table for queries about product ratings, customer feedback, 
        review trends, or product satisfaction analysis.
        """
    )
]

print("✓ Table schema objects created")
print(f"  Number of tables: {len(table_schema_objs)}")

✓ Table schema objects created
  Number of tables: 4


### 3.2 Build Vector Index of Table Schemas

In [17]:
# Create node mapping (maps table schemas to actual database tables)
table_node_mapping = SQLTableNodeMapping(sql_database)

# Build ObjectIndex with VectorStoreIndex
# This creates vector embeddings of table descriptions for semantic search
obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)

print("✓ ObjectIndex created")
print("  Table schemas are now indexed for semantic retrieval")

✓ ObjectIndex created
  Table schemas are now indexed for semantic retrieval


### 3.3 Test Table Retrieval

In [18]:
# Create a retriever
obj_retriever = obj_index.as_retriever(similarity_top_k=2)

# Test: What tables are relevant for different queries?
test_queries = [
    "What is the total revenue?",
    "How many customers do we have?",
    "What are the highest rated products?",
    "Show me products that are out of stock"
]

print("Testing table retrieval:\n")
for query in test_queries:
    print(f"Query: {query}")
    retrieved_objs = obj_retriever.retrieve(query)
    print(f"  Retrieved {len(retrieved_objs)} tables")
    
    # Try to show scores if available
    for i, obj in enumerate(retrieved_objs, 1):
        try:
            # Try different ways to access the score
            if hasattr(obj, 'score'):
                print(f"    Table {i}: score = {obj.score:.3f}")
            elif hasattr(obj, 'get_score'):
                print(f"    Table {i}: score = {obj.get_score():.3f}")
            else:
                print(f"    Table {i}: retrieved successfully")
        except:
            print(f"    Table {i}: retrieved successfully")
    print()

print("✓ Table retrieval is working!")
print("  The query engine will automatically use the most relevant tables")

Testing table retrieval:

Query: What is the total revenue?
  Retrieved 2 tables
    Table 1: retrieved successfully
    Table 2: retrieved successfully

Query: How many customers do we have?
  Retrieved 2 tables
    Table 1: retrieved successfully
    Table 2: retrieved successfully

Query: What are the highest rated products?
  Retrieved 2 tables
    Table 1: retrieved successfully
    Table 2: retrieved successfully

Query: Show me products that are out of stock
  Retrieved 2 tables
    Table 1: retrieved successfully
    Table 2: retrieved successfully

✓ Table retrieval is working!
  The query engine will automatically use the most relevant tables


## Section 4: SQLTableRetrieverQueryEngine

Now we'll use the query engine that automatically retrieves relevant tables.

### 4.1 Initialize Query Engine with Retriever

In [19]:
# Create query engine with automatic table retrieval
query_engine = SQLTableRetrieverQueryEngine(
    sql_database=sql_database,
    table_retriever=obj_retriever,
    llm=llm,
    verbose=True
)

print("✓ SQLTableRetrieverQueryEngine initialized")
print("  The engine will automatically retrieve relevant tables for each query")

✓ SQLTableRetrieverQueryEngine initialized
  The engine will automatically retrieve relevant tables for each query


### 4.2 Query Across Multiple Tables

In [20]:
# Query 1: Revenue analysis (uses orders table)
query = "What is the total revenue from all orders?"
print(f"Query: {query}")
print("=" * 70)

response = query_engine.query(query)
print(f"\nAnswer: {response}")
print("\n" + "=" * 70)

Query: What is the total revenue from all orders?
> Table Info: Table 'orders' has columns: order_id (BIGINT), customer_id (BIGINT), product_id (BIGINT), quantity (BIGINT), order_date (DATETIME), status (TEXT), order_total (FLOAT), . The table description is: 
        Contains order transactions with order_id, customer_id, product_id, quantity, 
        order_date, status, and order_total.
        Use this table for queries about sales, revenue, order volumes, order status,
        purchase patterns, or transaction analysis.
        
> Table Info: Table 'products' has columns: product_id (BIGINT), product_name (TEXT), category (TEXT), price (FLOAT), stock (BIGINT), . The table description is: 
        Contains product catalog with product_id, product_name, category, price, and stock.
        Use this table for queries about product information, pricing, inventory levels,
        product categories, or available products.
        
> Table desc str: Table 'orders' has columns: order_id (

In [21]:
# Query 2: Customer segmentation (uses customers table)
query = "How many customers are in each region?"
print(f"Query: {query}")
print("=" * 70)

response = query_engine.query(query)
print(f"\nAnswer: {response}")
print("\n" + "=" * 70)

Query: How many customers are in each region?
> Table Info: Table 'customers' has columns: customer_id (BIGINT), name (TEXT), email (TEXT), region (TEXT), signup_date (DATETIME), . The table description is: 
        Contains customer information including customer_id, name, email, region, and signup_date.
        Use this table for queries about customer demographics, contact information, 
        customer segmentation by region, or customer acquisition over time.
        
> Table Info: Table 'orders' has columns: order_id (BIGINT), customer_id (BIGINT), product_id (BIGINT), quantity (BIGINT), order_date (DATETIME), status (TEXT), order_total (FLOAT), . The table description is: 
        Contains order transactions with order_id, customer_id, product_id, quantity, 
        order_date, status, and order_total.
        Use this table for queries about sales, revenue, order volumes, order status,
        purchase patterns, or transaction analysis.
        
> Table desc str: Table 'custome

In [18]:
# Query 3: Product analysis (uses products table)
query = "Which product category has the most products?"
print(f"Query: {query}")
print("=" * 70)

response = query_engine.query(query)
print(f"\nAnswer: {response}")
print("\n" + "=" * 70)

Query: Which product category has the most products?
> Table Info: Table 'products' has columns: product_id (BIGINT), product_name (TEXT), category (TEXT), price (FLOAT), stock (BIGINT), . The table description is: 
        Contains product catalog with product_id, product_name, category, price, and stock.
        Use this table for queries about product information, pricing, inventory levels,
        product categories, or available products.
        
> Table Info: Table 'reviews' has columns: review_id (BIGINT), product_id (BIGINT), customer_id (BIGINT), rating (BIGINT), review_date (DATETIME), . The table description is: 
        Contains product reviews with review_id, product_id, customer_id, rating, and review_date.
        Use this table for queries about product ratings, customer feedback, 
        review trends, or product satisfaction analysis.
        
> Table desc str: Table 'products' has columns: product_id (BIGINT), product_name (TEXT), category (TEXT), price (FLOAT), st

In [22]:
# Query 4: Product reviews (uses reviews table)
query = "What is the average product rating?"
print(f"Query: {query}")
print("=" * 70)

response = query_engine.query(query)
print(f"\nAnswer: {response}")
print("\n" + "=" * 70)

Query: What is the average product rating?
> Table Info: Table 'reviews' has columns: review_id (BIGINT), product_id (BIGINT), customer_id (BIGINT), rating (BIGINT), review_date (DATETIME), . The table description is: 
        Contains product reviews with review_id, product_id, customer_id, rating, and review_date.
        Use this table for queries about product ratings, customer feedback, 
        review trends, or product satisfaction analysis.
        
> Table Info: Table 'products' has columns: product_id (BIGINT), product_name (TEXT), category (TEXT), price (FLOAT), stock (BIGINT), . The table description is: 
        Contains product catalog with product_id, product_name, category, price, and stock.
        Use this table for queries about product information, pricing, inventory levels,
        product categories, or available products.
        
> Table desc str: Table 'reviews' has columns: review_id (BIGINT), product_id (BIGINT), customer_id (BIGINT), rating (BIGINT), review_

In [23]:
# Query 5: Complex query requiring JOIN
query = "Show me the top 5 best-selling products by quantity sold"
print(f"Query: {query}")
print("=" * 70)

response = query_engine.query(query)
print(f"\nAnswer: {response}")
print("\n" + "=" * 70)

Query: Show me the top 5 best-selling products by quantity sold
> Table Info: Table 'products' has columns: product_id (BIGINT), product_name (TEXT), category (TEXT), price (FLOAT), stock (BIGINT), . The table description is: 
        Contains product catalog with product_id, product_name, category, price, and stock.
        Use this table for queries about product information, pricing, inventory levels,
        product categories, or available products.
        
> Table Info: Table 'orders' has columns: order_id (BIGINT), customer_id (BIGINT), product_id (BIGINT), quantity (BIGINT), order_date (DATETIME), status (TEXT), order_total (FLOAT), . The table description is: 
        Contains order transactions with order_id, customer_id, product_id, quantity, 
        order_date, status, and order_total.
        Use this table for queries about sales, revenue, order volumes, order status,
        purchase patterns, or transaction analysis.
        
> Table desc str: Table 'products' has col

In [24]:
# Query 6: Multi-table analysis
query = "Which region has generated the most revenue?"
print(f"Query: {query}")
print("=" * 70)

response = query_engine.query(query)
print(f"\nAnswer: {response}")
print("\n" + "=" * 70)

Query: Which region has generated the most revenue?
> Table Info: Table 'customers' has columns: customer_id (BIGINT), name (TEXT), email (TEXT), region (TEXT), signup_date (DATETIME), . The table description is: 
        Contains customer information including customer_id, name, email, region, and signup_date.
        Use this table for queries about customer demographics, contact information, 
        customer segmentation by region, or customer acquisition over time.
        
> Table Info: Table 'orders' has columns: order_id (BIGINT), customer_id (BIGINT), product_id (BIGINT), quantity (BIGINT), order_date (DATETIME), status (TEXT), order_total (FLOAT), . The table description is: 
        Contains order transactions with order_id, customer_id, product_id, quantity, 
        order_date, status, and order_total.
        Use this table for queries about sales, revenue, order volumes, order status,
        purchase patterns, or transaction analysis.
        
> Table desc str: Table 'c

### 4.3 Compare with Direct Table Specification

Let's compare the automatic retrieval approach with manually specifying tables.

In [25]:
# Manual approach (from Notebook 1)
from llama_index.core.query_engine import NLSQLTableQueryEngine

# Must specify all tables upfront
manual_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["customers", "products", "orders", "reviews"],  # All tables
    llm=llm
)

print("Manual approach: All table schemas sent to LLM every time")
print("Automatic approach: Only relevant tables retrieved per query")
print("\nFor 4 tables, the difference is small.")
print("For 50+ tables, automatic retrieval is essential!")

Manual approach: All table schemas sent to LLM every time
Automatic approach: Only relevant tables retrieved per query

For 4 tables, the difference is small.
For 50+ tables, automatic retrieval is essential!


## Section 5: Bonus - DuckDB for Direct CSV Queries

### 5.1 Why DuckDB?

**DuckDB** is an in-process SQL OLAP database perfect for analytical workloads. While we use SQLite for LlamaIndex compatibility, DuckDB excels at:
- Querying CSV/Parquet files directly (no loading required!)
- Fast analytical queries
- OLAP workloads
- Data analysis pipelines

Let's demonstrate DuckDB's powerful CSV querying capabilities!

In [26]:
# Create a DuckDB engine for CSV queries
duckdb_engine = create_engine("duckdb:///:memory:")

print("✓ DuckDB engine created for CSV queries")
print("  DuckDB can query CSV files without loading them into tables!")

✓ DuckDB engine created for CSV queries
  DuckDB can query CSV files without loading them into tables!


### 5.2 Query CSV Files Directly with DuckDB

DuckDB's killer feature: query CSV files as if they were tables!

In [27]:
# DuckDB can query CSV files without loading them!
with duckdb_engine.connect() as conn:
    result = conn.execute(text("""
        SELECT 
            category,
            COUNT(*) as product_count,
            ROUND(AVG(price), 2) as avg_price
        FROM read_csv_auto('products.csv')
        GROUP BY category
        ORDER BY product_count DESC
    """))
    
    print("Query results from CSV (no table loading required!):")
    print()
    for row in result:
        print(f"  {row[0]}: {row[1]} products, avg price ${row[2]}")

Query results from CSV (no table loading required!):

  Electronics: 13 products, avg price $255.31
  Books: 10 products, avg price $216.78
  Clothing: 9 products, avg price $277.04
  Food: 9 products, avg price $177.37
  Home: 9 products, avg price $195.58


### 5.3 Complex Analytical Queries with DuckDB

DuckDB is optimized for analytical queries. Let's run a complex multi-table JOIN directly on CSV files!

In [28]:
# Complex analytical query joining multiple CSV files
import time

analytical_query = """
SELECT 
    p.category,
    COUNT(DISTINCT o.order_id) as order_count,
    SUM(o.order_total) as total_revenue,
    ROUND(AVG(o.order_total), 2) as avg_order_value
FROM read_csv_auto('orders.csv') o
JOIN read_csv_auto('products.csv') p ON o.product_id = p.product_id
GROUP BY p.category
ORDER BY total_revenue DESC
"""

start = time.time()
with duckdb_engine.connect() as conn:
    result = pd.read_sql(analytical_query, conn)
end = time.time()

print(f"✓ Query execution time: {(end - start) * 1000:.2f} ms")
print(f"  (Querying 2 CSV files with JOIN - no table loading!)\n")
print("Results:")
print(result)

✓ Query execution time: 39.79 ms
  (Querying 2 CSV files with JOIN - no table loading!)

Results:
      category  order_count  total_revenue  avg_order_value
0  Electronics          134      169412.22          1264.27
1     Clothing           85      127252.07          1497.08
2        Books          108      120766.69          1118.21
3         Home           96       90907.28           946.95
4         Food           77       57402.43           745.49


### 5.4 When to Use SQLite vs DuckDB

**Use SQLite when:**
- Need LlamaIndex SQLDatabase integration
- Write-heavy workloads
- Embedded applications
- Need ACID guarantees
- Smaller datasets

**Use DuckDB when:**
- Analytical queries (aggregations, complex JOINs)
- Querying CSV/Parquet files directly
- Large datasets
- Read-heavy workloads
- Data analysis pipelines
- Want to avoid loading data into tables

**Best Practice:** Use SQLite for text-to-SQL with Llama Index (better compatibility), and DuckDB for direct data analysis on files!

## Section 6: Exercises

### Exercise 1: Basic Queries
1. How many products are currently out of stock?
2. What is the total number of orders by status?
3. Which customer has placed the most orders?

### Exercise 2: Table Retrieval Analysis
1. Test the table retriever with different queries
2. Observe which tables are retrieved for each query type
3. Try queries that might require multiple tables

### Exercise 3: Complex Analytics
1. Find the most profitable product category
2. Calculate customer lifetime value by region
3. Identify products with high ratings but low sales

### Exercise 4: DuckDB Exploration
1. Write a DuckDB query to join all 4 CSV files
2. Compare query performance between SQLite (loaded tables) vs DuckDB (CSV files)
3. Find use cases where DuckDB's direct CSV querying would be beneficial

In [ ]:
# Your code for Exercise 2

In [ ]:
# Your code for Exercise 1

In [ ]:
# Your code for Exercise 3

In [ ]:
# Your code for Exercise 4

## Summary

In this notebook, you learned:

✓ Working with multi-table databases  
✓ Dynamic table retrieval with `ObjectIndex`  
✓ `SQLTableRetrieverQueryEngine` for automatic table selection  
✓ When to use dynamic retrieval vs direct specification  
✓ DuckDB's powerful CSV querying capabilities  
✓ Choosing between SQLite and DuckDB for different use cases

## Key Takeaways

1. **Scalability**: Dynamic table retrieval is essential for databases with many tables
2. **Context matters**: Good table descriptions improve retrieval accuracy
3. **Database choice**: SQLite for LlamaIndex compatibility, DuckDB for direct file queries
4. **Vector search**: Semantic similarity helps find relevant tables

## Next Steps

Continue to **Notebook 03: Advanced Text-to-SQL with Workflows** to learn:
- LlamaIndex Workflows architecture
- Query-time row retrieval
- Production-ready patterns
- Advanced error handling

---

**Excellent work!** You're now ready for advanced topics. 🚀